In [1]:
import pandas as pd

x_cor=pd.read_csv("coordinates.csv")


from sklearn.ensemble import RandomForestClassifier

forrest_model = RandomForestClassifier(random_state=1)


import numpy as np
y=x_cor.Letter
x = np.array(x_cor.drop(["Letter"],axis=1))
print(x)

forrest_model.fit(x,y)

print(forrest_model.predict([x[1300]]))



[[0.29282656 0.69724166 0.49932077 ... 0.36460924 0.30919102 0.41618434]
 [0.27692923 0.70581979 0.50404412 ... 0.38585556 0.29179573 0.47934836]
 [0.31792128 0.76113606 0.54396707 ... 0.42765641 0.29895112 0.51451778]
 ...
 [0.58569735 0.78788805 0.67309117 ... 0.56536031 0.41611534 0.59038919]
 [0.6133135  0.74924445 0.68836933 ... 0.57118177 0.43614194 0.59975714]
 [0.6444217  0.72169828 0.7084353  ... 0.59352577 0.45129201 0.61295831]]
['C']


In [2]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

hands = mp_hands.Hands(
    min_detection_confidence=0.7, min_tracking_confidence=0.5)

while cap.isOpened():
  success, image = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
    continue

  # Flip the image horizontally for a later selfie-view display, and convert
  # the BGR image to RGB.
  image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
  # To improve performance, optionally mark the image as not writeable to
  # pass by reference.
  image.flags.writeable = False
  results = hands.process(image)
  
  # Draw the hand annotations on the image.
  image.flags.writeable = True
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  
  if results.multi_hand_landmarks:
    points=[]
    for hand_landmarks in results.multi_hand_landmarks:
      for i in range(21):
          x=hand_landmarks.landmark[i].x
          y=hand_landmarks.landmark[i].y
          points.append(x)
          points.append(y)  
    # print(points)
    if len(points)==42:
            predict=forrest_model.predict([points])
    #print(predict)    
            
    for hand_landmarks in results.multi_hand_landmarks:
      mp_drawing.draw_landmarks(
          image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
  if results.multi_hand_landmarks:
        
      cv2.putText(image, '%s' % (predict), (100,400), cv2.FONT_HERSHEY_SIMPLEX, 4, (255,255,255), 4)
  cv2.imshow('MediaPipe Hands', image)
  if cv2.waitKey(5) & 0xFF == 27:
    break
